In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
import numpy as np

In [38]:
spark = SparkSession \
.builder \
.appName("Python Spark SQL basic example") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()

sc = spark.sparkContext

(a)

In [72]:
def create_dataframe(fileName):
    schema_price = ["Price", "Timestamp", "InstanceType",
                    "ProductDescription", "AvailabilityZone"]
    lines = sc.textFile(fileName)
    header = lines.first()
    lines = lines.filter(lambda l :not str(l).startswith(header)) # drop header
    # drop the first col in RDD
    parts = lines.map(lambda l: l.split("\t")[1:]).filter(lambda l: len(l) == len(schema_price))
    
    parts = parts.map(lambda p: [p[i].strip() for i in range(len(schema_price))])
    
    fields = [StructField(field_name, StringType(), True) for field_name in schema_price]
    schema = StructType(fields)
    
    df_price = spark.createDataFrame(parts, schema_price)
    df_price = df_price.withColumn("Price", col("Price").cast(FloatType()))
    df_price = df_price.withColumn("Timestamp", to_timestamp("Timestamp", "yyyy-MM-dd'T'HH:mm:ssZ"))

    return df_price

In [73]:
filename = "dataset-EC2-series/prices-eu-central-1-2019-05-24.txt.gz"
df = create_dataframe(filename)
print(df.count())
df.show(20)

106340
+-------+-------------------+------------+------------------+----------------+
|  Price|          Timestamp|InstanceType|ProductDescription|AvailabilityZone|
+-------+-------------------+------------+------------------+----------------+
| 0.2656|2019-05-24 03:26:23|  m5.2xlarge|        SUSE Linux|   eu-central-1c|
| 0.1656|2019-05-24 03:26:23|  m5.2xlarge|        Linux/UNIX|   eu-central-1c|
| 0.9046|2019-05-24 03:25:54| c5n.9xlarge|        SUSE Linux|   eu-central-1a|
| 0.8046|2019-05-24 03:25:54| c5n.9xlarge|        Linux/UNIX|   eu-central-1a|
|23.2913|2019-05-24 03:25:50|x1e.32xlarge|           Windows|   eu-central-1a|
| 0.1702|2019-05-24 03:25:47|   c5.xlarge|        SUSE Linux|   eu-central-1c|
| 0.0702|2019-05-24 03:25:47|   c5.xlarge|        Linux/UNIX|   eu-central-1c|
| 0.2326|2019-05-24 03:16:57| c5n.2xlarge|        SUSE Linux|   eu-central-1b|
| 0.1326|2019-05-24 03:16:57| c5n.2xlarge|        Linux/UNIX|   eu-central-1b|
| 2.0056|2019-05-24 03:09:48|  d2.8xlarge|   

(b)

In [81]:
df.groupBy(['InstanceType', df.ProductDescription]).agg(avg("Price").alias("avg_price"))\
            .sort("avg_price", ascending=False).show(50)

+------------+------------------+------------------+
|InstanceType|ProductDescription|         avg_price|
+------------+------------------+------------------+
|x1e.32xlarge|           Windows| 41.29813147179874|
|x1e.32xlarge|        SUSE Linux| 35.45298683352587|
|x1e.32xlarge|        Linux/UNIX|35.352987929088314|
| p3.16xlarge|           Windows|28.689710946083068|
| p3.16xlarge|        SUSE Linux|25.567980808577513|
| p3.16xlarge|        Linux/UNIX| 25.46798058683649|
| x1.32xlarge|           Windows|19.008055222709224|
|x1e.16xlarge|           Windows| 18.74707821100065|
| p2.16xlarge|           Windows| 17.91937599834214|
|x1e.16xlarge|        SUSE Linux|15.755645188630796|
|x1e.16xlarge|        Linux/UNIX|15.655646125475565|
| p2.16xlarge|        SUSE Linux| 15.07459573664217|
| p2.16xlarge|        Linux/UNIX|14.974595607855381|
| x1.32xlarge|        SUSE Linux|13.192031178651032|
| x1.32xlarge|        Linux/UNIX|13.092031014407123|
|  p3.8xlarge|           Windows|12.9439632824